In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义 PINN 网络
class PINN(nn.Module):
    def __init__(self, layers):
        """
        参数:
            layers: 一个列表，定义各层的神经元个数，例如 [1, 20, 20, 20, 1]
        """
        super(PINN, self).__init__()
        self.activation = nn.Tanh()
        layer_list = []
        for i in range(len(layers)-2):
            layer_list.append(nn.Linear(layers[i], layers[i+1]))
            layer_list.append(self.activation)
        # 最后一层不使用激活函数
        layer_list.append(nn.Linear(layers[-2], layers[-1]))
        self.net = nn.Sequential(*layer_list)
    
    def forward(self, x):
        return self.net(x)

# 定义 PINN 训练器
class PINNTrainer:
    def __init__(self, model, nu, lr=1e-3, device='cpu'):
        """
        参数:
            model: PINN 模型
            nu: 物理参数（粘性系数）
            lr: 学习率
            device: 训练设备，例如 'cpu' 或 'cuda'
        """
        self.device = device
        self.model = model.to(device)
        self.nu = nu
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.loss_func = nn.MSELoss()

    def pde_residual(self, x):
        """
        计算 PDE 残差:
            u * u_x - nu * u_xx
        参数:
            x: 输入的 collocation 点 (需要计算梯度)
        """
        # 将 x 设置到对应设备并开启梯度
        x = x.to(self.device)
        x.requires_grad_(True)
        u = self.model(x)
        # 计算一阶导数 u_x
        u_x = torch.autograd.grad(u, x,
                                  grad_outputs=torch.ones_like(u),
                                  retain_graph=True,
                                  create_graph=True)[0]
        # 计算二阶导数 u_xx
        u_xx = torch.autograd.grad(u_x, x,
                                   grad_outputs=torch.ones_like(u_x),
                                   retain_graph=True,
                                   create_graph=True)[0]
        # 一维稳态 Burgers 方程: u * u_x - nu * u_xx = 0
        residual = u * u_x - self.nu * u_xx
        return residual

    def train(self, collocation_points, boundary_points, boundary_values, epochs=10000):
        """
        训练函数
        参数:
            collocation_points: 用于计算 PDE 残差的点（内部点）
            boundary_points: 边界点
            boundary_values: 对应的边界条件值
            epochs: 训练的迭代次数
        """
        collocation_points = collocation_points.to(self.device)
        boundary_points = boundary_points.to(self.device)
        boundary_values = boundary_values.to(self.device)
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            # 计算 collocation 点处的 PDE 残差
            res = self.pde_residual(collocation_points)
            pde_loss = self.loss_func(res, torch.zeros_like(res))
            # 计算边界条件损失
            u_b = self.model(boundary_points)
            bc_loss = self.loss_func(u_b, boundary_values)
            # 总损失
            loss = pde_loss + bc_loss
            loss.backward()
            self.optimizer.step()
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Total Loss: {loss.item()}, PDE Loss: {pde_loss.item()}, BC Loss: {bc_loss.item()}")

# 示例：训练求解一维稳态 Burgers 方程
if __name__ == "__main__":
    # 物理参数：粘性系数 (例如 nu = 0.01/π)
    nu = 0.01 / torch.pi
    # 定义神经网络结构：输入层1个神经元，三个隐藏层（每层20个神经元），输出层1个神经元
    layers = [1, 20, 20, 20, 1]
    model = PINN(layers)
    trainer = PINNTrainer(model, nu, lr=1e-3, device='cpu')

    # 构造 collocation 点（用于计算 PDE 残差）
    N_collocation = 1000
    # 在区间 [0, 1] 上均匀采样
    x_collocation = torch.linspace(0, 1, N_collocation).view(-1, 1)

    # 定义边界点和边界条件（这里假设 u(0)=0, u(1)=1，可根据问题实际设置）
    x_boundary = torch.tensor([[0.0], [1.0]])
    u_boundary = torch.tensor([[0.0], [1.0]])

    # 开始训练
    trainer.train(x_collocation, x_boundary, u_boundary, epochs=10000)


Epoch 0, Total Loss: 0.8894960880279541, PDE Loss: 0.00016058312030509114, BC Loss: 0.8893355131149292
Epoch 10, Total Loss: 0.4934851825237274, PDE Loss: 6.827798415542929e-07, BC Loss: 0.4934844970703125
Epoch 20, Total Loss: 0.23382219672203064, PDE Loss: 0.0016793678514659405, BC Loss: 0.232142835855484
Epoch 30, Total Loss: 0.1624755561351776, PDE Loss: 0.017759229987859726, BC Loss: 0.1447163224220276
Epoch 40, Total Loss: 0.15193217992782593, PDE Loss: 0.03733969107270241, BC Loss: 0.11459248512983322
Epoch 50, Total Loss: 0.12618932127952576, PDE Loss: 0.03381095454096794, BC Loss: 0.09237836301326752
Epoch 60, Total Loss: 0.11776483058929443, PDE Loss: 0.03379520773887634, BC Loss: 0.08396962285041809
Epoch 70, Total Loss: 0.11165773868560791, PDE Loss: 0.04670729115605354, BC Loss: 0.06495044380426407
Epoch 80, Total Loss: 0.11050085723400116, PDE Loss: 0.056190039962530136, BC Loss: 0.054310817271471024
Epoch 90, Total Loss: 0.1102554202079773, PDE Loss: 0.05429616943001747,

KeyboardInterrupt: 